# Intro

In this notebook, we will learn how to use hail functions to identify MNVs in a given vcf or matrixtable.

Most of the work in this paper is performed in google cloud using hail. Specifically, we ran 
```
cluster start mycluster --num-preemptible-workers 8 --vep --init gs://gnomad-public/tools/inits/master-init.sh
```
Followed by 
```
cluster connect mycluster notebook
```
To build the jupyter notebook in google cloud cluster. 

After building the cluster and accessing this notebook, 
the first step is to import hail and relevant tools:





In [1]:
import hail as hl
import hail.expr.aggregators as agg
from typing import *


If you have trouble building the environment, we recommend to go through the [Hail intallation tutorials](https://hail.is/docs/0.2/getting_started.html). 

Also we use [cloudtools](https://github.com/Nealelab/cloudtools) for starting and managing the clusters.


Now assuming that we have the environment built, 
first, let's start with an example of 1000genomes (subset 9831 variants in a chunk of chr1, in 2504 samples): 

In [2]:
vcf = hl.import_vcf("gs://gnomad-public/MNV/head100M_1000Genomes_chr1.vcf", call_fields=["GT"])
vcf.describe()


Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.2.3
SparkUI available at http://10.128.0.44:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.11-61a2083d5773
LOGGING: writing to /home/hail/hail-20190309-2255-0.2.11-61a2083d5773.log


----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        CIEND: array<int32>, 
        CIPOS: array<int32>, 
        CS: str, 
        END: int32, 
        IMPRECISE: bool, 
        MC: array<str>, 
        MEINFO: array<str>, 
        MEND: int32, 
        MLEN: int32, 
        MSTART: int32, 
        SVLEN: array<int32>, 
        SVTYPE: str, 
        TSD: str, 
        AC: array<int32>, 
        AF: array<float64>, 
        NS: int32, 
        AN: int32, 
        EAS_AF: array<float64>, 
        EUR_AF: array<float64>, 
        AFR_AF: array<float64>, 
        AMR_AF: array<float64>, 
        SAS_AF: array<float64>, 
        DP: int32, 
        AA: str, 
        VT: array<str>, 
        EX_TARGET: bool, 
        MUL

Although the imported vcf would naturally take the form of matrixtable in hail, 
let's re-write and read as a matrixtable (as in [hail tutorial](https://hail.is/docs/0.2/tutorials/01-genome-wide-association-study.html))

Also, since we would like to split multi allelic sites for convenience, we are also applying `split_multi_hts` here.

(Replace thr directory with your own, unless you are actually using `gnomad-qingbowang` as your favorite directory name.)

In [4]:
vcf = hl.split_multi_hts(vcf)
vcf.write('gs://gnomad-qingbowang/MNV/head100M_1000Genomes_chr1.mt', overwrite=True)
mt = hl.read_matrix_table('gs://gnomad-qingbowang/MNV/head100M_1000Genomes_chr1.mt')
mt.describe()

2019-03-09 23:03:38 Hail: INFO: wrote matrix table with 9884 rows and 2504 columns in 2 partitions to gs://gnomad-qingbowang/MNV/head100M_1000Genomes_chr1.mt


----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {
        CIEND: array<int32>, 
        CIPOS: array<int32>, 
        CS: str, 
        END: int32, 
        IMPRECISE: bool, 
        MC: array<str>, 
        MEINFO: array<str>, 
        MEND: int32, 
        MLEN: int32, 
        MSTART: int32, 
        SVLEN: array<int32>, 
        SVTYPE: str, 
        TSD: str, 
        AC: array<int32>, 
        AF: array<float64>, 
        NS: int32, 
        AN: int32, 
        EAS_AF: array<float64>, 
        EUR_AF: array<float64>, 
        AFR_AF: array<float64>, 
        AMR_AF: array<float64>, 
        SAS_AF: array<float64>, 
        DP: int32, 
        AA: str, 
        VT: array<str>, 
        EX_TARGET: bool, 
        MUL

## Cleaning
This toy example is easy, but in real application the matrixtable could be large. 
We will through away some unnecessary fields at this stage. 
(If you want to keep some more fields, change the lines as needed.)

In [5]:
mt = mt.select_cols() #dropping unneeded  columns makes things faster
mt = mt.annotate_rows(AC = mt.info.AC[mt.a_index-1], AF = mt.info.AF[mt.a_index-1]) #for case of multiallelic
mt = mt.select_rows(mt.filters, mt.AC, mt.AF) #or any rows that you want to store for future investigation

mt = mt.filter_entries(hl.is_defined(mt.GT) & mt.GT.is_non_ref()) #interested in non-ref only.

Before calling MNV, let's inspect the data we are looking at:
What we want in the `mt` data is, a pair of variants whose sample `s` is the same, and `GT` has a non_ref variant in same haplotype.

In [6]:
mt.GT.show()

+---------------+------------+-----------+------+
| locus         | alleles    | s         | GT   |
+---------------+------------+-----------+------+
| locus<GRCh37> | array<str> | str       | call |
+---------------+------------+-----------+------+
| 1:10177       | ["A","AC"] | "HG00096" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00097" | 0|1  |
| 1:10177       | ["A","AC"] | "HG00099" | 0|1  |
| 1:10177       | ["A","AC"] | "HG00100" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00101" | 0|0  |
| 1:10177       | ["A","AC"] | "HG00102" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00103" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00105" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00106" | 1|0  |
| 1:10177       | ["A","AC"] | "HG00107" | 0|0  |
+---------------+------------+-----------+------+
showing top 10 rows



Thanks to the `window_by_locus` function, this could be archieved relatively easily:

## Running window_by_locus

In [7]:
mt = hl.window_by_locus(mt, 2) #partition in window -- we only care within codon reading frame, so the max distance is set to be 2

Now you can see each entrie is tagged with `prev_entry`, which contains the information of the variant within the window range, 
in the same individual (if there is).

In [8]:
mt.describe() #now we have this prev_rows / prev_entries

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh37>
    'alleles': array<str>
    'filters': set<str>
    'AC': int32
    'AF': float64
    'prev_rows': array<struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        filters: set<str>, 
        AC: int32, 
        AF: float64
    }>
----------------------------------------
Entry fields:
    'GT': call
    'prev_entries': array<struct {
        GT: call
    }>
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [9]:
mt.prev_entries.show() #This is how the prev_entry looks like

+---------------+------------+-----------+-------------------------+
| locus         | alleles    | s         | prev_entries            |
+---------------+------------+-----------+-------------------------+
| locus<GRCh37> | array<str> | str       | array<struct{GT: call}> |
+---------------+------------+-----------+-------------------------+
| 1:10177       | ["A","AC"] | "HG00096" | []                      |
| 1:10177       | ["A","AC"] | "HG00097" | []                      |
| 1:10177       | ["A","AC"] | "HG00099" | []                      |
| 1:10177       | ["A","AC"] | "HG00100" | []                      |
| 1:10177       | ["A","AC"] | "HG00101" | []                      |
| 1:10177       | ["A","AC"] | "HG00102" | []                      |
| 1:10177       | ["A","AC"] | "HG00103" | []                      |
| 1:10177       | ["A","AC"] | "HG00105" | []                      |
| 1:10177       | ["A","AC"] | "HG00106" | []                      |
| 1:10177       | ["A","AC"] | "HG

Actually most of the case there is no any prev_entries for a variant. So we can filter down the number a lot at this stage:

In [10]:
mt = mt.filter_entries((mt.prev_entries.length() > 0))
mt = mt.filter_entries((hl.is_defined(mt.GT) & (mt.prev_entries.length() > 0))) #throwing away no MNV SNPs
mt = mt.filter_entries(mt.prev_entries.filter(lambda x: x.GT.is_non_ref()).length() > 0) #same


Now you can see the non null prev_entrie clearer:

In [11]:
mt.prev_entries.show() #now we have the mnv candidate

+---------------+------------+-----------+-------------------------+
| locus         | alleles    | s         | prev_entries            |
+---------------+------------+-----------+-------------------------+
| locus<GRCh37> | array<str> | str       | array<struct{GT: call}> |
+---------------+------------+-----------+-------------------------+
| 1:10506       | ["C","G"]  | "HG02315" | [(0|1)]                 |
| 1:13118       | ["A","G"]  | "HG00097" | [(1|0)]                 |
| 1:13118       | ["A","G"]  | "HG00101" | [(1|0)]                 |
| 1:13118       | ["A","G"]  | "HG00105" | [(1|0)]                 |
| 1:13118       | ["A","G"]  | "HG00107" | [(0|1)]                 |
| 1:13118       | ["A","G"]  | "HG00111" | [(0|1)]                 |
| 1:13118       | ["A","G"]  | "HG00121" | [(1|0)]                 |
| 1:13118       | ["A","G"]  | "HG00123" | [(1|0)]                 |
| 1:13118       | ["A","G"]  | "HG00128" | [(0|1)]                 |
| 1:13118       | ["A","G"]  | "HG

Now we are going to explode the matrixtable, and from here on we are going to work with the `table` format, 
a bit more intuitive for users working with pandas, r dataframe etc.

In [21]:
#explode it to entries
et = mt.key_cols_by().entries() # Matrix with 1000 rows (variant) + 1000 cols (sample)=> 1 million entries
et = et.annotate(indices = hl.range(0, hl.len(et.prev_rows)))
et = et.explode('indices') #for the case where there are more than one prev_row for a variant
et = et.transmute(prev_row = et.prev_rows[et.indices],
                      prev_entry = et.prev_entries[et.indices])#tracking back the one with matched indices
et = et.annotate(dist=et.locus.position - et.prev_row.locus.position) #annotating the distance

#filtering
et = et.filter(et.dist>0) #distance=0 is just multiallelic
et = et.filter( (et.alleles[0].length()==1) & (et.alleles[1].length()==1) \
                 & (et.prev_row.alleles[0].length()==1) & (et.prev_row.alleles[1].length()==1) )#interested in SNP only
et = et.filter((et.filters.length()==0) & (et.prev_row.filters.length()==0)) #if you are interested in filter pass variants only

et.show()

+---------------+------------+----------+-------+----------+-----------+------+
| locus         | alleles    | filters  |    AC |       AF | s         | GT   |
+---------------+------------+----------+-------+----------+-----------+------+
| locus<GRCh37> | array<str> | set<str> | int32 |  float64 | str       | call |
+---------------+------------+----------+-------+----------+-----------+------+
| 1:10506       | ["C","G"]  | {}       |     1 | 2.00e-04 | "HG02315" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00097" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00101" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00105" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00107" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00111" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00121" | 1|0  |
| 1:13118       | ["A","G"]  | {}       

In [22]:
et.count()

153304

## Phase inspection

(See figure 1 a, b for the basic information)

Now there are 153,304 variant pairs. Still a lot.
Of course not all of them are MNV. Now let's look at MNVs. 
MNVs of 2 bp can be classified into 3 configurations (assuming diploid):

1. Both are heterozygote
2. One of them are heterozygote
3. Both are homozygote

In case of 2, and 3, we don't need to look at the phase information. 1 is homozygous MNV, and 2 is heterozygous MNV. 
The case 3 is a bit tricky, you actually need to look at the phase information to see whether they are in same haplotype (*cis*) or not.

(Warning: In this example the variants do not have the phase ID `PID`, but in many cases there are, as in gnomAD. 
We recommend users to carefully define the right condition to call MNVs.)


In [23]:
et = et.annotate(hom = ((et.GT.is_diploid()) & (et.prev_entry.GT.is_diploid()) & et.GT.is_hom_var() & (et.prev_entry.GT.is_hom_var())),
                 hethom = ((et.GT.is_diploid()) & (et.prev_entry.GT.is_diploid()) & (et.GT.is_hom_var() & et.prev_entry.GT.is_het_ref()) | (et.GT.is_het_ref() & et.prev_entry.GT.is_hom_var())),#including hom-het, just not distinguishing them two.
                 hethet = ((et.GT.is_diploid()) & (et.prev_entry.GT.is_diploid()) & (et.GT.phased&et.prev_entry.GT.phased) & (et.GT.is_het_ref()&et.prev_entry.GT.is_het_ref()) & (et.GT==et.prev_entry.GT) ))

In [28]:
#now you can for example look at the het het pairs
et_het = et.filter(et.hethet)
et_het.show()
et_het.write("gs://gnomad-qingbowang/MNV/demo_et_het.ht", overwrite=True) #and we are going to save them

+---------------+------------+----------+-------+----------+-----------+------+
| locus         | alleles    | filters  |    AC |       AF | s         | GT   |
+---------------+------------+----------+-------+----------+-----------+------+
| locus<GRCh37> | array<str> | set<str> | int32 |  float64 | str       | call |
+---------------+------------+----------+-------+----------+-----------+------+
| 1:10506       | ["C","G"]  | {}       |     1 | 2.00e-04 | "HG02315" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00097" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00101" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00105" | 1|0  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00107" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00111" | 0|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00121" | 1|0  |
| 1:13118       | ["A","G"]  | {}       

2019-03-09 23:34:34 Hail: INFO: wrote table with 11721 rows in 2 partitions to gs://gnomad-qingbowang/MNV/demo_et_het.ht


In [29]:
et_hethom = et.filter(et.hethom)
et_hethom.show()
et_hethom.write("gs://gnomad-qingbowang/MNV/demo_et_hethom.ht", overwrite=True)

+---------------+------------+----------+-------+----------+-----------+------+
| locus         | alleles    | filters  |    AC |       AF | s         | GT   |
+---------------+------------+----------+-------+----------+-----------+------+
| locus<GRCh37> | array<str> | set<str> | int32 |  float64 | str       | call |
+---------------+------------+----------+-------+----------+-----------+------+
| 1:55165       | ["A","G"]  | {}       |     1 | 2.00e-04 | "NA18505" | 1|0  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "HG01886" | 1|1  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "HG02571" | 1|1  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "HG03086" | 1|1  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "HG03115" | 1|1  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "NA19031" | 1|1  |
| 1:362905      | ["T","G"]  | {}       |   633 | 1.26e-01 | "NA19095" | 1|1  |
| 1:362905      | ["T","G"]  | {}       

2019-03-09 23:34:58 Hail: INFO: wrote table with 1184 rows in 2 partitions to gs://gnomad-qingbowang/MNV/demo_et_hethom.ht


In [30]:
et_hom = et.filter(et.hom)
et_hom.show()
et_hom.write("gs://gnomad-qingbowang/MNV/demo_et_hom.ht", overwrite=True)

+---------------+------------+----------+-------+----------+-----------+------+
| locus         | alleles    | filters  |    AC |       AF | s         | GT   |
+---------------+------------+----------+-------+----------+-----------+------+
| locus<GRCh37> | array<str> | set<str> | int32 |  float64 | str       | call |
+---------------+------------+----------+-------+----------+-----------+------+
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00261" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00265" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00328" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00382" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG00640" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG01085" | 1|1  |
| 1:13118       | ["A","G"]  | {}       |   486 | 9.70e-02 | "HG01161" | 1|1  |
| 1:13118       | ["A","G"]  | {}       

2019-03-09 23:35:35 Hail: INFO: wrote table with 11589 rows in 2 partitions to gs://gnomad-qingbowang/MNV/demo_et_hom.ht


## aggregation per variant
In the analysis of the mechanisms and the impact in the population, what we want to know is the allele count of MNV, 
rather than the individuals who are carrying it. In this case, we can use aggregator in hail to get per variant statistics:

In [32]:
per_variant_het = et_het.group_by('locus', 'alleles', "prev_row").aggregate(n=hl.agg.count()) 
per_variant_hom = et_hom.group_by('locus', 'alleles', "prev_row").aggregate(n=hl.agg.count()) 
per_variant_hethom = et_hethom.group_by('locus', 'alleles', "prev_row").aggregate(n=hl.agg.count()) 


and it is not easy to annotate back some columns that does not depend on the variant, for downstream analysis.
(e.g. in this tutorial we threw away filtered variants, but in gnomAD analysis we kept them and compared the statistics between filtered and unfilted variants. Just for such case, we are annotating the `filters` column as well, although it is not directly used in this tutorial)

(Supplementary Figure 11 was generated in such way)

In [36]:
#and we also rename some columns for convenience
et_het = et_het.key_by("locus", "alleles", "prev_row")
per_variant_het = per_variant_het.annotate(dist = et_het[per_variant_het.key].dist,
                                                  AF = et_het[per_variant_het.key].AF,
                                                  AC = et_het[per_variant_het.key].AC,
                                                  filters = et_het[per_variant_het.key].filters)
per_variant_het = per_variant_het.annotate(prev_locus = per_variant_het.prev_row.locus,
                                                  prev_alleles = per_variant_het.prev_row.alleles,
                                                  prev_filters = per_variant_het.prev_row.filters,
                                                  prev_AC = per_variant_het.prev_row.AC,
                                                  prev_AF = per_variant_het.prev_row.AF)
et_hom = et_hom.key_by("locus", "alleles", "prev_row")
per_variant_hom = per_variant_hom.annotate(dist = et_hom[per_variant_hom.key].dist,
                                                  AF = et_hom[per_variant_hom.key].AF,
                                                  AC = et_hom[per_variant_hom.key].AC,
                                                  filters = et_hom[per_variant_hom.key].filters)
per_variant_hom = per_variant_hom.annotate(prev_locus = per_variant_hom.prev_row.locus,
                                                  prev_alleles = per_variant_hom.prev_row.alleles,
                                                  prev_filters = per_variant_hom.prev_row.filters,
                                                  prev_AC = per_variant_hom.prev_row.AC,
                                                  prev_AF = per_variant_hom.prev_row.AF)
et_hethom = et_hethom.key_by("locus", "alleles", "prev_row")
per_variant_hethom = per_variant_hethom.annotate(dist = et_hethom[per_variant_hethom.key].dist,
                                                  AF = et_hethom[per_variant_hethom.key].AF,
                                                  AC = et_hethom[per_variant_hethom.key].AC,
                                                  filters = et_hethom[per_variant_hethom.key].filters)
per_variant_hethom = per_variant_hethom.annotate(prev_locus = per_variant_hethom.prev_row.locus,
                                                  prev_alleles = per_variant_hethom.prev_row.alleles,
                                                  prev_filters = per_variant_hethom.prev_row.filters,
                                                  prev_AC = per_variant_hethom.prev_row.AC,
                                                  prev_AF = per_variant_hethom.prev_row.AF)


In [37]:
#check how it looks like:
per_variant_het.show()

2019-03-09 23:40:46 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:40:54 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:02 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:18 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:34 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:42 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:41:50 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+------------+----------------+------------------+------------------+
| locus         | alleles    | prev_row.locus | prev_row.alleles | prev_row.filters |
+---------------+------------+----------------+------------------+------------------+
| locus<GRCh37> | array<str> | locus<GRCh37>  | array<str>       | set<str>         |
+---------------+------------+----------------+------------------+------------------+
| 1:10506       | ["C","G"]  | 1:10505        | ["A","T"]        | {}               |
| 1:13118       | ["A","G"]  | 1:13116        | ["T","G"]        | {}               |
| 1:48328       | ["A","T"]  | 1:48327        | ["C","A"]        | {}               |
| 1:49989       | ["A","G"]  | 1:49988        | ["T","A"]        | {}               |
| 1:51049       | ["A","C"]  | 1:51047        | ["A","T"]        | {}               |
| 1:51050       | ["A","T"]  | 1:51049        | ["A","C"]        | {}               |
| 1:55545       | ["C","T"]  | 1:55544        | ["T","

Now the table is much cleaner. The final step is to combine the frequency of het, hethom and hom.
This is also easily done in hail by using `join`:

In [38]:
comb = per_variant_het.key_by("locus", "alleles","prev_locus","prev_alleles","dist","AF","AC","filters","prev_AF","prev_AC","prev_filters") \
            .join(per_variant_hom.key_by("locus", "alleles", "prev_locus", "prev_alleles", "dist", "AF", "AC", "filters", "prev_AF","prev_AC", "prev_filters"), how='outer') \
            .join(per_variant_hethom.key_by("locus", "alleles","prev_locus","prev_alleles","dist","AF","AC","filters","prev_AF","prev_AC","prev_filters"), how='outer')
comb = comb.transmute(n_hethet=hl.or_else(comb.n, 0), n_hethom=hl.or_else(comb.n_1, 0), n_homhom=hl.or_else(comb.n_2, 0))
comb = comb.select("n_hethet","n_hethom","n_homhom")
comb = comb.annotate(n_total = comb.n_hethet + comb.n_hethom + comb.n_homhom)
comb.show()

2019-03-09 23:42:51 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'prev_row' -> 'prev_row_1'
    'n' -> 'n_1'
2019-03-09 23:42:51 Hail: INFO: Table.join: renamed the following fields on the right to avoid name conflicts:
    'prev_row' -> 'prev_row_2'
    'n' -> 'n_2'
2019-03-09 23:43:41 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:43:50 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:43:58 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:07 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:15 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:23 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:39 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:44:47 Hail: INFO: Ordering unsorted 

+---------------+------------+---------------+--------------+-------+----------+-------+
| locus         | alleles    | prev_locus    | prev_alleles |  dist |       AF |    AC |
+---------------+------------+---------------+--------------+-------+----------+-------+
| locus<GRCh37> | array<str> | locus<GRCh37> | array<str>   | int32 |  float64 | int32 |
+---------------+------------+---------------+--------------+-------+----------+-------+
| 1:10506       | ["C","G"]  | 1:10505       | ["A","T"]    |     1 | 2.00e-04 |     1 |
| 1:13118       | ["A","G"]  | 1:13116       | ["T","G"]    |     2 | 9.70e-02 |   486 |
| 1:48328       | ["A","T"]  | 1:48327       | ["C","A"]    |     1 | 4.39e-03 |    22 |
| 1:49989       | ["A","G"]  | 1:49988       | ["T","A"]    |     1 | 5.79e-03 |    29 |
| 1:51049       | ["A","C"]  | 1:51047       | ["A","T"]    |     2 | 1.60e-03 |     8 |
| 1:51050       | ["A","T"]  | 1:51049       | ["A","C"]    |     1 | 1.60e-03 |     8 |
| 1:55165       | ["A

## aggregation per sample

If users are more interested in getting per sample statistics, it is possible to do so as well, using the same principle:

In [39]:
per_sample_het = et_het.group_by("s").aggregate(n=hl.agg.count()) 
per_sample_het.show()

2019-03-09 23:59:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-09 23:59:30 Hail: INFO: Ordering unsorted dataset with network shuffle


+-----------+-------+
| s         |     n |
+-----------+-------+
| str       | int64 |
+-----------+-------+
| "HG00096" |     2 |
| "HG00097" |     3 |
| "HG00099" |     3 |
| "HG00100" |     1 |
| "HG00101" |     5 |
| "HG00103" |     2 |
| "HG00105" |     5 |
| "HG00106" |     5 |
| "HG00107" |     3 |
| "HG00108" |     3 |
+-----------+-------+
showing top 10 rows



This tells that sample "HG00096" has 2 heterozygous MNVs, and so on.
(Or in practice, you can join the `et_het`, `et_hom`, and `et_hethom` first, and then do the aggregation)
(This kind of approach would be more interesting after annotating the functional impact of each MNV.)

Alternatively, we can keep the sample names tagged with each MNV entries, if we are more interested in the individual samples:
(Warning: since some MNVs have allele frequency ~=1, this could make your file very large depending on the sample size and allele frequency distributions)


In [41]:
per_variant_het = et_het.group_by('locus', 'alleles', "prev_row").aggregate(n=hl.agg.count(), s=hl.agg.collect(et_het.s))
per_variant_het.show()

2019-03-10 00:03:54 Hail: INFO: Ordering unsorted dataset with network shuffle
2019-03-10 00:04:00 Hail: INFO: Ordering unsorted dataset with network shuffle


+---------------+------------+----------------+------------------+------------------+
| locus         | alleles    | prev_row.locus | prev_row.alleles | prev_row.filters |
+---------------+------------+----------------+------------------+------------------+
| locus<GRCh37> | array<str> | locus<GRCh37>  | array<str>       | set<str>         |
+---------------+------------+----------------+------------------+------------------+
| 1:10506       | ["C","G"]  | 1:10505        | ["A","T"]        | {}               |
| 1:13118       | ["A","G"]  | 1:13116        | ["T","G"]        | {}               |
| 1:48328       | ["A","T"]  | 1:48327        | ["C","A"]        | {}               |
| 1:49989       | ["A","G"]  | 1:49988        | ["T","A"]        | {}               |
| 1:51049       | ["A","C"]  | 1:51047        | ["A","T"]        | {}               |
| 1:51050       | ["A","T"]  | 1:51049        | ["A","C"]        | {}               |
| 1:55545       | ["C","T"]  | 1:55544        | ["T","

## End notes

- This tutorial explained how the MNV discovery works. As a user, of course you do not really need to run through this pipeline step by step everytime, since we have prepared a function `get_mnv.py`, which by default writes per_variant files and other intermediate files. 
(You can also let the function write per individual stats, with the argument `per_indv=True`, or keep the sample info for the per variant statistics by `keep_samplenames=True`.)

- However, since the use case of MNV call could be different from one lab to another, we have not made the function so flexible by default. 
(e.g. it does not look at `PID` by default, which is wrong if your data does have `PID`.)
We hope the users to use the function as a prototype to exlore their own analysis. 

- (Also, we have not explained about MNV consisting of 3bp (or more). This is because we have learned that such MNV are extremely rare, and both the probability that it could be causal for phenotype of a single person and the effect on the population level statistics is limited. However, interested users can look at the `tnv_call.py` to apply such analysis.)

- The next step is to annotate the MNVs, to see which MNV has interesting impact on the transcript. see you at `annotate_mnv.ipynb`.
